Long-term create a jupyter kernel for any REPL interface

https://github.com/n-riesco/nel/blob/master/lib/nel.js

https://github.com/n-riesco/jp-kernel/blob/master/lib/jp-kernel.js

https://github.com/takluyver/bash_kernel/blob/master/bash_kernel/kernel.py

https://github.com/n-riesco/ijavascript/blob/d459956f76a22f9ec89937e02645ee7555f92d2b/lib/kernel.js

https://github.com/notablemind/jupyter-nodejs/blob/1388e1957792ef167ccfc7a9cf6c64c3950ded76/lib/kernel.js

https://github.com/JuliaLang/IJulia.jl

https://github.com/n-riesco/jp-babel/blob/master/lib/kernel.js

looks like it is just passed to the command line in rust

https://github.com/google/evcxr/blob/602db3ef52bfddeb34608877bd72b9d0d112fa26/evcxr/src/module.rs


Fix ijavascript not showing filename

https://nodejs.org/api/vm.html#vm_class_vm_script

Fix the problem with ijavascript not supporting %%

Go through this list and find demo code for every language and make sure it works with our REPL, minimize dependencies

https://github.com/jupyter/jupyter/wiki/Jupyter-kernels

https://jupyter-client.readthedocs.io/en/stable/kernels.html

https://jupyter-client.readthedocs.io/en/stable/wrapperkernels.html


Add HTML, CSS/SCSS kernels with some sort of visual output specially make for the file type

Create a kernels notebook





Goal: rewrite entire nel, jupyter-nodejs, jp-babel, jp-kernel functionality in one notebook and a few lines of code

Goal: transpile the kernel in to the native language using AST translations

TODO: implement both meta kernels in python and socket based kernels (through library include in another language or through command like repl interface), translate zmq to every language

TODO: use syntax highlighter to "train" a transpiler to write socket based kernels in any language, at least 
BROWSER C# GO JAVA NODE.JS PHP PYTHON RUBY from Google

TODO: make a processing kernel https://github.com/processing/p5.js/wiki/p5.js,-node.js,-socket.io


get kernel json?


In [ ]:
var path = require('path');

function kernelJsonTemplate(kernel_json) {
    return {
        argv,
        display_name,
        language,
        metadata,
        env,
        interrupt_mode
    } = kernel_json
}

function standardJsonTemplate(kernel_json) {
    return kernelJsonTemplate(Object.assign({
//        argv: ['python', '-m', kernel_json.path, '-f', '{connection_file}']
        argv: ['npm', 'run', 'import',
               '--', 'get javascript kernel', '["{connection_file}"]'],
    }, kernel_json))
}

module.exports = {
    kernelJsonTemplate,
    standardJsonTemplate
};


get meta kernel?


In [ ]:

function kernelMetaTemplate(meta_kernel) {
    return {
        do_execute,
        do_complete,
        do_inspect,
        do_history,
        do_is_complete,
        do_shutdown
    } = meta_kernel
}

function javascriptMeta(meta_kernel) {
    return Object.assign({
        do_init: (request, kernel) => {
            process.on('message', (message) => kernel[message[0]].apply(kernel, message[1]))
        },
        do_execute: (code) => new require('vm').Script(transpile(code)),
        do_shutdown: () => {},
        do_is_complete: (request, kernel) => {
            try {
                kernel.do_execute(request);
                return true;
            } catch {
                return false;
            }
        },
        do_complete: (request) => {
            // TODO: call lang-server code
            throw new Error('not implemented!');
        },
        do_history: (request) => {
            throw new Error('not implemented!');
        },
        do_inspect: (request) => {
            // TODO: call lang-server code
            throw new Error('not implemented!');
        }
    }, meta_kernel)
}

module.exports = {
    kernelMetaTemplate,
    javascriptMeta
};


get kernel language?


In [ ]:

function languageTemplate(language_info) {
    return {
        mimetype,
        name,
        file_extension,
        version,
        pygments_lexer,
        codemirror_mode,
        nbconvert_exporter
    } = language_info
}

module.exports = languageTemplate;


 get kernel info?
 

In [ ]:

function kernelInfoTemplate(kernel_info) {
    return {
        protocol_version,
        implementation,
        implementation_version,
        banner,
        language_info,
        help_links
    } = kernel_info
}

module.exports = kernelInfoTemplate;


get kernel session?


In [ ]:

function kernelSessionTemplate(session) {
    return {
        init,
        message,
        respond,
        display,
        execute_request,
        inspect_request,
        complete_request,
        history_request,
        kernel_info_request,
        is_complete_request,
        connect_request,
        comm_info_request,
        kernel_info_request,
        shutdown_request,
        interrupt_request,
        input_request
    } = session
}

var count = 0;
function standardMessaging(session) {
    return Object.assign({
        init: (config, kernel, session) => {
            var process = require("child_process")
                .spawn(process.argv[0], ['--eval', kernel], {
                    cwd: config.cwd || '.',
                    stdio: [process.stdin, process.stdout, process.stderr, 'ipc']
                })
            // TODO: translate this to a call
            process.on('message', session.message);
            return setupSockets(config).then(sockets => {
                sockets.heartbeat.on('message', message => sockets.heartbeat.send(message));
                sockets.control.on('message', message => session.shutdown_request(message));
                sockets.shell.on('message', message => session.message(message));
                // TODO: finish this
                sockets.stdin.on('message', message => session.input_request(message));
            })
        },
        message: (respond, message, kernel, session) => {
            if(content.clear) respond({clear_output: content.clear})
            if(content.input) respond({input_request: content.input})
            if(typeof session[msg.header.msg_type] === 'undefined') {
                throw new Error(`unhandled message type ${msg.header.msg_type}`);
            }
            session[msg.header.msg_type](content);
        },
        respond: (respond, message, kernel, session, sockets) => {
            if(Object.keys(content)[0] === 'shutdown_reply') sockets.control.send(message);
            // TODO: shellSocket if _reply
            
            else sockets.iopub.send(message);
        },
        display: (respond, request) => {
            respond(request.display_id
                    ? {update_display_data: {metadata: {}, data: request.mime, transient: request.display_id}}
                    : {display_data: {metadata: {}, data: request.mime}})
        },
        input_request: (respond, request) => {
            //this.onReplies[response.header.msg_id] = onReply;
        },
        shutdown_request: (respond, request, kernel, process) => {
            if(request.header.msg_type !== 'shutdown_request') {
                throw new Error(`unhandled message type ${request.header.msg_type}`);
            }
            respond({status: {execution_state: 'busy'}});
            return Promise.resolve()
                .then(() => process.kill('SIGTERM'))
                .then(() => request.content.restart
                      ? kernel.init()
                      : void 0)
                .then(() => respond({shutdown_reply: request.content}))
                .then(() => respond({status: {execution_state: 'idle'}}))
        },
        kernel_info_request: (respond, request, kernel) => {
            respond({status: {execution_state: 'busy'}});
            respond({kernel_info_reply: kernelInfoTemplate(kernel_info)})
            respond({status: {execution_state: 'idle'}});
        },
        is_complete_request: (respond, request, kernel) => {
            respond({status: {execution_state: 'busy'}});
            respond({is_complete_reply: {
                status: kernel.do_is_complete(request, session)
                ? 'complete': 'incomplete',
                indent: ''
            }});
            respond({status: {execution_state: 'idle'}});
        },
        execute_request: (respond, request, kernel) => {
            var executionCount = ++count;
            respond({status: {execution_state: 'busy'}});
            respond({execute_input: {
                execution_count: executionCount,
                code: request.content.code,
            }})
            var result;
            return Promise.resolve()
                .then(() => kernel.do_execute(request))
                .then(r => result = r)
                .then(() => respond({execute_reply: {
                    status: 'ok',
                    execution_count: executionCount,
                    payload: [], // TODO(NR) not implemented,
                    user_expressions: {}, // TODO(NR) not implemented,
                }}))
                .then(() => respond({status: {execution_state: 'idle'}}))
                .then(() => !result.execute_result.data
                      ? void 0
                      : respond({execute_result: {
                        execution_count: executionCount,
                        data: result.mime,
                        metadata: {},
                    }}))
        },
        complete_request: (respond, request, kernel) => {
            respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_complete(request))
                .then(r => result = r)
                .then(() => respond({status: {execution_state: 'idle'}}))
                .then(() => respond({complete_reply: {
                    matches: result.completion.list,
                    cursor_start: result.completion.cursorStart,
                    cursor_end: result.completion.cursorEnd,
                    status: "ok",
                }}))
        },
        history_request: (respond, request, kernel) => {
            respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_history(request))
                .then(r => result = r)
                .then(() => respond({status: {execution_state: 'idle'}}))
                .then(() => respond({history_reply: {
                    history: [] // TODO
                }}))
        },
        inspect_request: (respond, request, kernel) => {
            var executionCount = ++count;
            respond({status: {execution_state: 'busy'}});
            var result;
            return Promise.resolve()
                .then(() => kernel.do_inspect(request))
                .then(r => result = r)
                .then(() => respond({status: {execution_state: 'idle'}}))
                .then(() => respond({inspect_reply: {
                    found: true,
                    data: {
                        /*
    if (result.inspection) {
        docstring = (
            result.inspection.type + ": " + result.inspection.string
        );
    }

    if (result.doc) {
        docstring = result.doc.usage;
        if (result.doc.usage) {
            docstring += "\n\n" + result.doc.description;
        }
    }*/
                        'text/plain': docstring,
                        'text/html': `<pre>${docstring}</pre>`,
                    },
                    metadata: {},
                    status: "ok",
                }}))
                .catch(e => respond({inspect_reply: {
                    status: "error",
                    execution_count: executionCount,
                    ename: result.error.ename,
                    evalue: result.error.evalue,
                    traceback: result.error.traceback,
                }}))
        }
    }, session);
}

module.exports = {
    standardMessaging,
    kernelSessionTemplate
};


get kernel class?


In [ ]:
var importer = require('../Core');
var languageTemplate = importer.import('get kernel language');
var kernelInfoTemplate = importer.import('get kernel info');
var {kernelMetaTemplate} = importer.import('get meta kernel');
var {kernelSessionTemplate} = importer.import('get kernel session');

function kernelTemplate({
    kernel_info,
    meta_kernel,
    language_info,
    session,
    install
}) {
    return {
        ...kernelInfoTemplate({...kernel_info, language_info}),
        ...kernelMetaTemplate(meta_kernel),
        language_info: languageTemplate(language_info),
        session: kernelSessionTemplate(session),
        install
    }
}

module.exports = kernelTemplate;


get javascript kernel?


In [5]:
var importer = require('../Core');
var {setupSockets, installKernel} = importer.import('bind to jupyter zmq sockets');
var kernelTemplate = importer.import('get kernel class');
var {standardMessaging} = importer.import('get kernel session');
var {javascriptMeta} = importer.import('get meta kernel');

var PACKAGE_VERSION = require('../package.json').version;

function getVersion(str) {
    return str.split('.').map(v => parseInt(v, 10)).join('.')
}

/*
var transpile = require("@babel/core").transform.bind({
    presets: [
        [require.resolve("@babel/preset-env"), {
            loose: true,
            targets: {node: true},
        }],
    ],
});
*/

function javascriptKernel(config) {
    var kernel = kernelTemplate({
        kernel_info: {
            protocol_version: 5.1,
            implementation: 'javascript',
            implementation_version: PACKAGE_VERSION,
            banner: 'NodeJS',
            // TODO: automatically create this from installation intructions
            help_links: ['https://nodejs.org']
        },
        language_info: {
            name: 'javascript',
            version: getVersion(process.versions.node),
            file_extension: '.js',
            mimetype: 'application/javascript'
        },
        session: standardMessaging({}),
        meta_kernel: javascriptMeta({}),
        install: installKernel
    })
    if(config === 'install') {
        kernel.install({
            display_name: kernel.banner,
            path: process.argv[2],
            language: kernel.implementation + '_test'
        });
    } else if(typeof config === 'string') {
        config = JSON.parse(fs.readFileSync(config));
        kernel.session.init(config, kernel.meta_kernel, kernel.session);
    }
    return kernel;
}

module.exports = javascriptKernel;

if(typeof $$ !== 'undefined') {
    $$.sendResult(javascriptKernel());
}



reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[5] aka get kernel class
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[2] aka get kernel language
reading notebook /Users/briancullinan/jupyter_ops/Core/kernels.ipynb
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[3] aka get kernel info
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[1] aka get meta kernel
compiling /Users/briancullinan/jupyter_ops/Core/kernels.ipynb[4] aka get kernel session


{ protocol_version: 5.1,
  implementation: 'javascript',
  implementation_version: undefined,
  banner: 'NodeJS',
  help_links: [ 'https://nodejs.org' ],
  language_info:
   { name: 'javascript',
     version: '10.16.0',
     file_extension: '.js',
     mimetype: 'application/javascript' },
  do_init: [Function: do_init],
  do_execute: [Function: do_execute],
  do_shutdown: [Function: do_shutdown],
  do_is_complete: [Function: do_is_complete],
  do_complete: [Function: do_complete],
  do_history: [Function: do_history],
  do_inspect: [Function: do_inspect],
  session:
   { init: [Function: init],
     message: [Function: message],
     respond: [Function: respond],
     display: [Function: display],
     input_request: [Function: input_request],
     shutdown_request: [Function: shutdown_request],
     kernel_info_request: [Function: kernel_info_request],
     is_complete_request: [Function: is_complete_request],
     execute_request: [Function: execute_request],
     complete_request:

bind to jupyter zmq sockets?


In [ ]:
var zmq = require("jmp").zmq;
var util = require('util');
var importer = require('../Core');
var {standardJsonTemplate} = importer.import('get kernel json');
var mkdirpSync = importer.import('mkdirp');

function setupSockets(config) {
  const sockets = {
    control: {
      port: config.control_port,
      type: 'xrep',
    },
    shell: {
      port: config.shell_port,
      type: 'xrep',
    },
    stdin: {
      port: config.stdin_port,
      type: 'router',
    },
    iopub: {
      port: config.iopub_port,
      type: 'pub',
    },
    heartbeat: {
      port: config.hb_port,
      type: 'rep',
    }
  }
  return Promise.all(sockets.map(s => setupSocket.apply(null, s, config)))
}

function setupSocket(config, general) {
  const sock = zmq.socket(config.type);
  const addr = general.transport + '://' + general.ip + ':' + config.port
  return util.promisify(sock.bind.bind(sock))(addr)
}

function installKernel(configJson) {
    console.log(`installing kernel ${JSON.stringify(configJson)}`);
    mkdirpSync(`./.kernel/${configJson.language}`);
    require('fs').writeFileSync(`./.kernel/${configJson.language}/kernel.json`,
                                JSON.stringify(standardJsonTemplate(configJson), null, 4));
    require('child_process').execSync(`jupyter kernelspec install --user --replace "./.kernel/${configJson.language}"`);
    require('rimraf').sync('./.kernel');
}

module.exports = {
    setupSockets,
    installKernel
};
